# Dealing with variable-length sequences efficiently

This notebook is meant to be run using a GPU.

RNNs can work with variable length sequences. This notebook shows how to do this efficiently with `keras`.

In [1]:

import tensorflow as tf
import numpy as np
import pandas

from os.path import exists
from tqdm.auto import tqdm

# since google colab has no wget
try:
    import wget
except:
    !pip install wget
    import wget

/ceph/lsowa/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


For this hands-on we will use the [Top Tagging dataset](https://arxiv.org/abs/1707.08966). This simplified MC data consists of signal Top jets and other quark and gluon background jets. For each jet, a maximum of 200 constituents are identified and ranked according their $P_T$. The four-momenta of these ranked constituents are now used as an input for our RNN. Note that the the number of constituents varies between the jets.



At first we will download the data and have a look at it.

In [2]:
if not exists("test.h5"):
    wget.download('https://desycloud.desy.de/index.php/s/llbX3zpLhazgPJ6/download?path=%2F&files=test.h5')

In [3]:
store = pandas.HDFStore("test.h5")
data = store.select("table", stop=1000) # Read the first X events

data = data.iloc[: , :-6] # drop the last six columns with truth information
data.head()

,E_0,PX_0,PY_0,PZ_0,E_1,PX_1,PY_1,PZ_1,E_2,PX_2,...,PY_197,PZ_197,E_198,PX_198,PY_198,PZ_198,E_199,PX_199,PY_199,PZ_199
436,218.364243,-172.341858,110.129105,-76.503624,153.661118,-111.320465,93.167969,-50.390713,76.708054,-56.523701,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
440,122.238762,26.738468,-91.613998,76.382225,121.227135,17.644758,-93.015450,75.715302,90.420105,21.377417,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
441,383.772308,-97.906456,79.640709,-362.426361,200.625992,-54.921326,37.994343,-189.184753,123.247223,-33.828953,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
444,132.492752,-77.763947,-87.322601,-62.304600,83.946594,-49.450481,-53.823605,-41.288010,28.072624,-19.964916,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
445,730.786987,-209.120010,-193.454315,-672.973877,225.477325,-75.363350,-66.226990,-201.926651,217.040192,-63.698189,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In order to have a dataset as if we measured it, we will remove all padded zeros from the dataset..

In [4]:
data = np.array(data)
data = [np.trim_zeros(data[i,:]) for i in range(data.shape[0])] # delete zeros and write jets in a list
data = [np.array([dat[0::4], dat[1::4], dat[2::4], dat[3::4]]).T for dat in data] # write features in an extra axis

Our dataset is now a list of measured jets consisting of arrays with shape (constituent, feature). Or more general: a list (batch) of arrays (sequence, feature). The number of constituents varies between the jets.

In [5]:
for jet in range(5):
    print(data[jet].shape[0], 'constituents for jet nr.', jet)

17 constituents for jet nr. 0
61 constituents for jet nr. 1
45 constituents for jet nr. 2
74 constituents for jet nr. 3
30 constituents for jet nr. 4


We want to feed this through the following 2-layer LSTM model - it takes a batch of sequences of arbitrary length and outputs a batch of numbers:

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(None, 4), return_sequences=True),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(1),
])

2022-08-05 10:40:34.766353: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-08-05 10:40:34.775035: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-08-05 10:40:34.775061: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-08-05 10:40:34.776359: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Libra

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 128)         68096     
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 199,809
Trainable params: 199,809
Non-trainable params: 0
_________________________________________________________________


How do we feed in the variable number of constituents? Well, since the first two input dimensions of our model are unspecified (jets, constituent) we can pass each jet separately. Let's see how fast this is.

In [8]:
for jet in tqdm(data):
    model(jet[np.newaxis, :])

100%|██████████| 1000/1000 [01:46<00:00,  9.36it/s]


Doesn't seem that bad does it?

Wait! We haven't seen yet how fast it could be ...

If you look at the GPU utilization (e.g. with `nvidia-smi`) while this is running you will see it is rather low. That's because RNNs are inherently sequential - we can't process the different steps of a sequence (of constituents in jet) in parallel.

But what we can do is to process each step of the sequence (each constituent in jet) in parallel across all jets in our batch!

Keras will do this if we provide batches that are Tensors of fixed length.

To try this out, let's enlarge the sequences to a fixed length and fill missing values with zeros:

In [9]:
padded_data = tf.keras.preprocessing.sequence.pad_sequences(data, padding="post", dtype="float32")

Now we have a dataset with a uniform number of constituents (again).

In [10]:
padded_data.shape # (batch, constituents, features)

(1000, 114, 4)

In [11]:
padded_data[0,:3,:] # four-momenta of the first three constituents in the first jet

array([[ 218.36424 , -172.34186 ,  110.129105,  -76.503624],
       [ 153.66112 , -111.320465,   93.16797 ,  -50.390713],
       [  76.70805 ,  -56.5237  ,   46.127293,  -23.695349]],
      dtype=float32)

Now evaluate the RNN, but this time parallelize the evaluation of the sequence (constituents) over the batch (jet) dimension.

In [12]:
model.predict(padded_data, batch_size=256, verbose=True)

4/4 [==============================] - 2s 243ms/step


array([[ 2.57577426e-07],
       [ 2.46610434e-05],
       [ 3.59926071e-06],
       [ 8.68468123e-05],
       [ 5.12861675e-07],
       [-5.16608225e-07],
       [ 3.90076515e-04],
       [ 2.45659839e-06],
       [ 8.69440730e-04],
       [ 9.49634682e-08],
       [ 2.11220026e-07],
       [ 9.10757171e-06],
       [ 6.49451904e-05],
       [ 1.56308431e-06],
       [ 5.88947501e-07],
       [-1.03999547e-07],
       [ 5.10440186e-05],
       [ 2.26313432e-06],
       [ 6.48514401e-07],
       [-7.36424699e-05],
       [-2.51332381e-07],
       [ 1.47006176e-06],
       [-4.05155092e-08],
       [ 1.75232867e-06],
       [ 6.46795613e-08],
       [ 4.03834889e-07],
       [ 2.15952969e-05],
       [ 3.73966077e-06],
       [ 6.33128138e-06],
       [ 1.80111711e-05],
       [ 2.67805626e-06],
       [ 1.01471596e-06],
       [ 4.75706656e-06],
       [ 4.24504151e-07],
       [ 3.29065006e-06],
       [ 4.01061607e-06],
       [ 5.82283405e-07],
       [ 1.17489719e-07],
       [ 3.4

That should have been **much** faster.

But now the model also processed the 0-padded values. We can see that e.g. the first output is different than what we expect from passing in the first sequence:

In [13]:
model(data[0][np.newaxis, :])

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.33475763]], dtype=float32)>

In keras we can solve this by a `Masking` layer - subsequent RNN layers will respect this and only process non-masked inputs.

For more info, see https://keras.io/guides/understanding_masking_and_padding/

In [14]:
masked_model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=0.0),
    tf.keras.layers.LSTM(128, input_shape=(None, 4), return_sequences=True),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(1),
])

In [15]:
masked_model.build(input_shape=(None, None, 4))

In [16]:
# set the weights such that we can compare the outputs of both models
masked_model.set_weights(model.get_weights())

In [17]:
masked_model.predict(padded_data, batch_size=256, verbose=True)

4/4 [==============================] - 4s 268ms/step


array([[-0.33475766],
       [-0.01731128],
       [-0.18375409],
       [-0.03383455],
       [-0.11857381],
       [-0.09039385],
       [-0.03079408],
       [-0.1682149 ],
       [-0.07121521],
       [-0.17601454],
       [-0.04293135],
       [-0.02554527],
       [-0.10789913],
       [-0.10809185],
       [-0.1260898 ],
       [-0.27116325],
       [-0.09162901],
       [-0.05189488],
       [-0.3692764 ],
       [-0.08113825],
       [-0.06828924],
       [-0.03039843],
       [-0.17266944],
       [-0.08109228],
       [-0.23863223],
       [-0.13923968],
       [-0.08068681],
       [-0.13825671],
       [-0.21249655],
       [-0.10877065],
       [-0.22970504],
       [-0.07365014],
       [-0.10772394],
       [-0.16475148],
       [-0.05700246],
       [-0.06233572],
       [-0.09431817],
       [-0.0758151 ],
       [-0.1338092 ],
       [-0.02640736],
       [-0.1352613 ],
       [-0.260996  ],
       [-0.01541425],
       [-0.19202007],
       [-0.05087405],
       [-0

This time the output is compatible with the one-by-one processing.